In [2]:
import os
import json
import warnings
import pandas as pd
warnings.simplefilter('ignore')

In [3]:
### load title name entity tags and short hand token
IMPROT_FILE = 'validData_Reduced.csv'
STANDARD_TITLE_FILES = ['tech_jobs.csv', 'non_tech_jobs.csv']
IMPORT_PATH = ''

In [4]:
### import golden source
# load cleaned profiles
df = pd.read_csv(os.path.join(IMPORT_PATH, IMPROT_FILE))
df['Start_Year'] = df['Start_Date'].apply(lambda x: int(x.split('-')[0]))
df['End_Year'] = df['End_Date'].apply(lambda x: int(x.split('-')[0]))
df.drop_duplicates(subset=['ID', 'End_Year'], keep='last', inplace=True)
df.drop_duplicates(subset=['ID', 'Start_Year'], keep='last', inplace=True)
org_employee_id_2_new_id = {id : "E" + str(idx + 1) for idx, id in enumerate(df.ID.unique())}
df['EmployeeID'] = df.ID.apply(lambda x: org_employee_id_2_new_id[x])
org_title_2_new_id = {id : "T" + str(idx + 1) for idx, id in enumerate(df.Title.unique())}
df['TitleID'] = df.Title.apply(lambda x: org_title_2_new_id[x])
df['Job_Index'] = df.groupby('EmployeeID').cumcount()
df.head(3)

,ID,Firm,Title,Start_Date,End_Date,Title_Group,EDU,Duration,DiversityScore,YrsOfExp,Start_Year,End_Year,EmployeeID,TitleID,Job_Index
0,"ACwAAA--S20BZncfI96Y51rtML5hkDoodwbFi-c,NAME_S...",Amazon,software engineer,2019-12-01,2021-10-01,software developer,"computer science, computer engineering, electr...",2,0.000000,1.0,2019,2021,E1,T1,0
1,"ACwAAA--zVABrailb54YsBsNL_ulhYNRVeX599Y,NAME_S...",Amazon,software engineer,2018-03-01,2019-03-01,software developer,"computer science, computer engineering, electr...",1,0.000000,0.0,2018,2019,E2,T1,0
2,"ACwAAA--zVABrailb54YsBsNL_ulhYNRVeX599Y,NAME_S...",JT4,technical analyst,2020-01-01,2020-07-01,it specialist,information technology,1,0.693147,1.0,2020,2020,E2,T2,1


In [5]:
### export employee/title id
with open(os.path.join(IMPORT_PATH, "title_id_dict.json"), "w") as outfile:
    json.dump(org_title_2_new_id, outfile)
with open(os.path.join(IMPORT_PATH, "employee_id_dict.json"), "w") as outfile:
    json.dump(org_employee_id_2_new_id, outfile)

In [16]:
### pick a yr
for yr in range(2008, 2022):
    df_cur = df[df.End_Year == yr].reset_index(drop=True)
    num = len(df_cur.EmployeeID.unique())
    print('There are {} made career transition.'.format(num))
yr = 2018
df_cur = df[df.End_Year == yr].reset_index(drop=True)
df_cur.head(3)

There are 4875 made career transition.
There are 4695 made career transition.
There are 6614 made career transition.
There are 7965 made career transition.
There are 9172 made career transition.
There are 808 made career transition.
There are 4779 made career transition.
There are 8115 made career transition.
There are 9515 made career transition.
There are 10502 made career transition.
There are 662 made career transition.
There are 4645 made career transition.
There are 14525 made career transition.
There are 11081 made career transition.


,ID,Firm,Title,Start_Date,End_Date,Title_Group,EDU,Duration,DiversityScore,YrsOfExp,Start_Year,End_Year,EmployeeID,TitleID,Job_Index
0,"ACwAAA-3B6YByaHpLMzxi_xgSx5z7yo4Jr9OoVw,NAME_S...",CheTariffa.it,business development manager,2018-03-01,2018-11-01,sales,"marketing, business administration, communicat...",1,1.475076,6.0,2018,2018,E10,T15,4
1,"ACwAAA-AHUMBdazFzMfVbCdF4q70jntKWTm29O4,NAME_S...",Salesforce,solutions engineer,2018-09-01,2018-12-01,software developer,"computer science, computer engineering, electr...",1,0.500402,4.0,2018,2018,E26,T43,2
2,"ACwAAA-J6kEBy1OTw4KstJs62oPGxeEwP92mmWw,NAME_S...",BaByliss Paris,junior product marketing associate,2018-01-01,2018-12-01,product marketers,"marketing, business administration",1,0.000000,0.0,2018,2018,E47,T69,0


In [17]:
### get the lookback window
w_size = 4 
window_s, window_e = yr - w_size, yr
cond1 = (df.End_Year >= window_s) & (df.End_Year <= window_e)
cond2 = (df.Start_Year >= window_s) & (df.Start_Year < window_e)
df_window = df[cond1|cond2].reset_index(drop=True)
df_window.head()

,ID,Firm,Title,Start_Date,End_Date,Title_Group,EDU,Duration,DiversityScore,YrsOfExp,Start_Year,End_Year,EmployeeID,TitleID,Job_Index
0,"ACwAAA-0QDgB49_y1aMPYEzyjrCBvC57YIbxuUo,NAME_S...",Microsoft,software engineer,2013-03-01,2015-08-01,software developer,"computer science, computer engineering, electr...",3,0.0,6.0,2013,2015,E4,T1,1
1,"ACwAAA-0QDgB49_y1aMPYEzyjrCBvC57YIbxuUo,NAME_S...",Microsoft,software engineer,2015-09-01,2016-04-01,software developer,"computer science, computer engineering, electr...",1,0.0,7.0,2015,2016,E4,T1,2
2,"ACwAAA-1ZzgB0vvmF7gL22NvzfUI4-hYU-BeTgE,NAME_S...",Amazon,chief administrative officer,2016-01-01,2017-02-01,administrative,"business administration, psychology",2,0.0,1.0,2016,2017,E6,T5,0
3,"ACwAAA-29-ABPtvmCpXbB6gKN0jihXHjTpqUfMw,NAME_S...",TripAdvisor,software engineer,2014-07-01,2017-11-01,software developer,"computer science, computer engineering, electr...",4,0.0,3.0,2014,2017,E7,T1,0
4,"ACwAAA-2AWwBnN5PdedLU3dU_29DxE1kUwsyqA8,NAME_S...",Amazon,technical support specialist,2014-08-01,2015-11-01,help desk,information technology,2,0.0,1.0,2014,2015,E8,T8,0


In [18]:
### ETE 
df_ete = df_cur[['EmployeeID', 'TitleID', 'Duration']].merge(df_window[['EmployeeID', 'TitleID', 'End_Year', 'Duration', 'Job_Index']], on='TitleID', how='left')
df_ete = df_ete[df_ete.EmployeeID_x != df_ete.EmployeeID_y]
df_ete

,EmployeeID_x,TitleID,Duration_x,EmployeeID_y,End_Year,Duration_y,Job_Index
1,E10,T15,1,E38,2017,1,2
2,E10,T15,1,E63,2016,1,1
3,E10,T15,1,E85,2017,2,1
4,E10,T15,1,E117,2015,1,0
5,E10,T15,1,E443,2014,1,3
...,...,...,...,...,...,...,...
476794,E29882,T131,1,E29160,2015,1,2
476795,E29882,T131,1,E29160,2016,1,3
476796,E29882,T131,1,E29233,2015,3,2
476797,E29882,T131,1,E29633,2015,3,0


In [21]:
### ETET DF
count = 0
df_col = []
gps = df_ete.groupby(['EmployeeID_x', 'TitleID'])
for id, gp in gps:    
    df_ref_tmp = df_window[df_window.EmployeeID.isin(gp.EmployeeID_y)]
    tmp = df_ref_tmp.merge(gp[['EmployeeID_y', 'End_Year', 'Duration_x', 'Duration_y', 'Job_Index']], left_on='EmployeeID', right_on='EmployeeID_y', how='left')
    tmp = tmp[(tmp.Start_Year >= tmp.End_Year_y)&(tmp.TitleID != id[1])][['EmployeeID_y', 'TitleID', 'Start_Year', 'End_Year_x', 'Duration', 'EDU', 'Duration_x', 'Duration_y', 'Job_Index_x']]
    tmp['Focal'], tmp['SharedTitle'] = id[0], id[1]
    df_col.append(tmp)
    count += 1
    print(count)
# clean up
df_agg = pd.concat(df_col, axis=0)
df_agg = df_agg[['Focal', 'SharedTitle', 'Duration_x', 'Duration_y', 'EmployeeID_y', 'TitleID', 'Start_Year', 'End_Year_x', 'Duration', 'Job_Index_x']].rename(
    columns={'EmployeeID_y' : 'Reference', 'TitleID' : 'TargetTitle', 'End_Year_x' : 'End_Year', 'Job_Index_x' : 'Job_Index'})

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [22]:
# reduce by sim (shared job worked similar yrs)
df_agg.reset_index(drop=True, inplace=True)
df_agg['sim'] = df_agg['Duration_x'] - df_agg['Duration_y']
df_agg['sim'] = df_agg['sim'].apply(abs)
df_agg_red = df_agg.loc[df_agg.groupby(['Focal', 'SharedTitle', 'TargetTitle']).sim.idxmin()]

In [25]:
df_agg_red[['Focal', 'SharedTitle', 'Reference', 'TargetTitle', 'Start_Year', 'End_Year', 'Duration', 'Job_Index']].to_csv(os.path.join(IMPORT_PATH, 'graph_etet_{yr}.csv'.format(yr=yr)), index=False)